In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import cv2
%matplotlib inline
from skimage.io import imread, imshow
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import time
import tensorflow as tf

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')
from patchify import patchify, unpatchify
import random
 
%matplotlib inline
import glob
from pathlib import Path
import math

seed = 42
np.random.seed = seed
from keras.utils.np_utils import normalize
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
from numpy import asarray

from keras.utils import normalize
from PIL import Image


In [ ]:
train_data_pth = "< PATH_TO_TRAIN_IMAGES >" 
train_mask_pth ="< PATH_TO_TRAIN_MASK >"

data_path_large = r'< PATH_TO_TEST_IMAGES >'   
lab_path_large = r'< PATH_TO_TEST_MASKS >'

In [ ]:
sum_pixels = 0
num_pixels = 0

# Loop over all images in the dataset directory
mean=[]
std = []
for filename in glob.glob(train_data_pth+"\*"):
    # Load the image
    img = cv2.imread( filename,0)/255.

    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Update the running sum of pixel values and the number of pixels
    mean.append(np.mean(img_array))
    std.append(np.std(img_array))
mean = np.mean(np.array(mean))
std = np.mean(np.array(std))
# Calculate the mean and standard deviation of the array of all images

# Print the calculated mean and standard deviation
print(mean,std)

In [ ]:
train_list = os.listdir(train_data_pth)

test_list_large = os.listdir(data_path_large)


In [1]:

def adjust_brightness_saturation_hue(image, brightness, saturation, hue):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image = hsv_image.astype(np.float32)

    hsv_image[..., 0] = (hsv_image[..., 0] + hue * 360) % 180
    hsv_image[..., 1] = np.clip(hsv_image[..., 1] * saturation, 0, 255)
    hsv_image[..., 2] = np.clip(hsv_image[..., 2] * brightness, 0, 255)

    hsv_image = hsv_image.astype(np.uint8)
    hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2GRAY)
    return hsv_image

def random_crop(image, label):
    y = np.random.randint(0,IMG_HEIGHT - patch_size+1)
    x = np.random.randint(0,IMG_WIDTH - patch_size+1)
    im_crop = image[ y:y+patch_size, x:x+patch_size]
    lab_crop = label[ y:y+patch_size, x:x+patch_size]

    im_crop = cv2.resize(im_crop, (IMG_HEIGHT, IMG_WIDTH))
    lab_crop = cv2.resize(lab_crop, (IMG_HEIGHT, IMG_WIDTH))
    
    return im_crop, lab_crop

def flip_horz(image, label):
    image = cv2.flip(image, 1)
    label = cv2.flip(label, 1)
    
    return image, label

def flip_ver(image, label):
    image = cv2.flip(image, 0)
    label = cv2.flip(label, 0)
    return image, label

def add_noise(image, mean , std_dev):
    random_number = random.randint(0, std_dev)
    noise = np.random.normal(mean, random_number, image.shape)
    noisy_img = image + noise
    
    return noisy_img

def random_rotation(image, label, angle_range=(-15, 15)):
    # Generate a random rotation angle within the given range
    angle = random.uniform(angle_range[0], angle_range[1])

    # Get the image dimensions
    height, width = image.shape[:2]

    # Create rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1.0)

    # Rotate the image using the rotation matrix
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR)

    # Rotate the label using the rotation matrix
    rotated_label = cv2.warpAffine(label, rotation_matrix, (width, height), flags=cv2.INTER_NEAREST)

    return rotated_image, rotated_label
    

In [ ]:

import numpy as np
import os


def image_data_generator(data_files, data_dir, lab_dir,batch_size, data_aug = False, main_test=False, mean_ch = False):
    while True:
        # Shuffle the data directory
#         data_files = os.listdir(data_dir)
        np.random.shuffle(data_files)

        # Divide the data files into batches
        for i in range(0, len(data_files), batch_size):
            batch_files = data_files[i:i+batch_size]

            # Load and preprocess the images in the batch
            batch_images = []
            batch_labels = []
            for file in batch_files:
               
                try:
                    image = cv2.imread(data_dir + file, 0)
                    

                    if main_test:

                        mask = cv2.imread(lab_dir+'M'+file, cv2.IMREAD_COLOR)

                    else:
                        mask = cv2.imread(lab_dir+file, cv2.IMREAD_COLOR)
                    
                  
                    if data_aug:
                        if random.getrandbits(1):
                    # Apply brightness, saturation, and hue adjustments
                            image = adjust_brightness_saturation_hue(image, BRIGHTNESS, SATURATION, HUE)
                        if random.getrandbits(1):
                            image, mask = flip_horz(image, mask)
                        elif random.getrandbits(1):

                            image, mask = flip_ver(image, mask)  

    #                     if random.getrandbits(1):
    #                         image, mask = random_crop(image, mask)
                        if random.getrandbits(1):

                            image = add_noise(image, 0, 5)
                        if random.getrandbits(1):
                            image, mask = random_rotation(image, mask)




                    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))/255.
                    if per_image_norm:
                        image = (image - np.mean(image))/np.std(image)
                    else:
                        if mean_ch:
                            image = (image - mean)/std
                        else:
                            image = (image - mean)/std
                    image = Image.fromarray(image)
                    image = tf.convert_to_tensor(np.array(image))
    #                 image = normalize(np.array(image), axis=1).reshape(IMG_HEIGHT,IMG_WIDTH ,1)


                    mask = cv2.resize(mask, (IMG_HEIGHT, IMG_WIDTH))
                    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
                    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)/255.
                    mask = mask.reshape(IMG_HEIGHT,IMG_WIDTH ,1)



        #             mask = cv2.resize(mask, (IMG_HEIGHT, IMG_WIDTH))

                    batch_images.append(image)
                    batch_labels.append(mask)
                except:
                    pass

            # Convert the batch data to numpy arrays
            batch_images = np.array(batch_images)
            batch_labels = np.array(batch_labels)




            yield batch_images, batch_labels




## Sample Usage

In [ ]:
# Add these constants for brightness, saturation, and hue adjustments
BRIGHTNESS = 0.2
SATURATION = 0.3
HUE = 0.25

# Create empty arrays
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 1
step_size = 386
patch_sz = 512
include_full_size_img = False

per_image_norm = False

val_set = 0.1
t_list = train_list[0:int((1-val_set)*len(train_list))]
v_list = train_list[int((1-val_set)*len(train_list)):]
train_gen = image_data_generator(t_list,train_data_pth,train_mask_pth, batch_size=32, data_aug=True)
val_gen = image_data_generator(v_list,train_data_pth,train_mask_pth, batch_size=32, data_aug=False)


test_gen_large = image_data_generator(test_list_large,data_path_large,lab_path_large, batch_size=32, data_aug=False, main_test=True)